In [1]:
from database import fetch_all
from sqlalchemy import select
from sqlalchemy.orm import selectinload
from tables import ArticleTable, CommentTable, UserTable

In [2]:
for user in fetch_all(select(UserTable)):
    print(f"[{user.id}] {user.name}")

[098e9ed9-6d0f-4555-a869-fd97abc10eb4] Alice
[f14a938b-aa89-48af-a436-a91050e97fbb] Bob
[1e8b6572-f1c2-415c-a075-6ed44d1cda40] Carol


In [3]:
articles = fetch_all(
    select(ArticleTable)
    .where(ArticleTable.published_at.is_not(None))
    .order_by(ArticleTable.created_at.desc())
    .options(
        selectinload(ArticleTable.author),
        selectinload(ArticleTable.comments).selectinload(CommentTable.author),
    )
)

print(articles[0].comments[1].author.name)

Carol


JOIN するテーブル (上記の場合 Comment など) に対して `.where()` や `.order_by()` を指定することはできない模様。これが不便

## Pydantic モデルへの変換
FastAPI を想定し、`.model_dump(mode="json")` できる状態にする。

In [6]:
from uuid import UUID

from pydantic import BaseModel, ConfigDict


class User(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    id: UUID
    name: str


class Article(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    id: UUID
    title: str
    author: User | None = None
    comments: list[Comment] = []


class Comment(BaseModel):
    id: UUID
    body: str
    author: User
    article: Article

In [7]:
from database import fetch_first

user = User.model_validate(fetch_first(select(UserTable)))
print(user.model_dump(mode="json"))

{'id': '098e9ed9-6d0f-4555-a869-fd97abc10eb4', 'name': 'Alice'}


テーブル・フィールドをひとつひとつ定義すればできるが、

In [ ]:
articles = fetch_all(
    select(ArticleTable)
    .where(ArticleTable.published_at.is_not(None))
    .order_by(ArticleTable.created_at.desc())
    .options(
        selectinload(ArticleTable.author),
        selectinload(ArticleTable.comments).selectinload(CommentTable.author),
    )
)

print([Article.model_validate(article) for article in articles])  # => ValidationError

ValidationError: 2 validation errors for Article
comments.0
  Input should be a valid dictionary or instance of Comment [type=model_type, input_value=<tables.CommentTable object at 0xffff56dfb820>, input_type=CommentTable]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
comments.1
  Input should be a valid dictionary or instance of Comment [type=model_type, input_value=<tables.CommentTable object at 0xffff5471ca70>, input_type=CommentTable]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type

ネストしたデータに対してまで一括変換できるわけではない。

## Pydantic モデルの動的生成を試す

In [11]:
from sqlalchemy import inspect

m = inspect(ArticleTable)

print("# columns")
for col in m.columns:
    print(f"{col.key}: {col.type.python_type}")

print("# relationships")
for rel in m.relationships:
    print(
        f"{rel.key}: {rel.mapper.class_} {rel.uselist} {rel.direction.name} {rel.back_populates}"
    )

# columns
id: <class 'uuid.UUID'>
created_at: <class 'datetime.datetime'>
updated_at: <class 'datetime.datetime'>
author_id: <class 'uuid.UUID'>
title: <class 'str'>
body: <class 'str'>
published_at: <class 'datetime.datetime'>
# relationships
author: <class 'tables.UserTable'> False MANYTOONE articles
comments: <class 'tables.CommentTable'> True ONETOMANY article


SQLAlchemy モデルの情報は取得できるが、

In [12]:
from pydantic import create_model

Article = create_model("Article", id=int, title=str)
a = Article(id=1, title="aaa")
a.id  # => Unknown 型
a.title  # => Unknown 型

'aaa'

- テーブルモデルから Pydantic モデルを動的に生成しても、静的解析が使用できない
  - 型定義を事前に自動生成しておくアプローチにならざるを得ない
  - もはや SQLAlchemy 関係なく、「テーブル定義に基づいて partial 型を定義したい」なら必ずそうなる
- どうする？ →「テーブル構造 → Pydantic の partial を事前生成」する仕組みを考える
    - 近いことをやっている： [PydanticでUpdate用モデルを動的生成する：バリデーション継承＋Optional対応まで](https://zenn.dev/kicchan/articles/0012_llyssm_make_update_model)